# Automated Hyperparameter Optimization Training using WMLA API

#### Notebook created by Kelvin Lui, Nov 2020
&nbsp;
&nbsp;
&nbsp;
&nbsp;
&nbsp;

In this notebook, you will learn how to submit a model and dataset to the Watson Machine Learning Accelerator (WMLA) API to run Hyper Parameter Optimization (HPO). In this particular example, we will be using the Pytorch MNIST HPO model as our training model, inject hyperparameters for the sub-training during search and submit a tuning metric for better results, and then query for the best job results. This notebook runs on Python 3.6 or 3.7.


![options](https://github.com/IBM/wmla-learning-path/raw/master/shared-images/WMLA-RestAPI-Demo.png)



![SpectrumComputeFamily_Conductor-HorizontalColorWhite.png](https://raw.githubusercontent.com/IBM/wmla-learning-path/master/shared-images/hpo.png)


For this notebook you will use a model and dataset that have already been set up to leverage the API.  For details on the API see [API Documentation](https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/deeplearning.html) in the Knowledge Center (KC).

## Table of contents

1. [Setup](#setup)<br>

2. [Configuring environment and project details](#configure)<br>

3. [Health Check](#health)<br>

4. [Training with the HPO API](#train)<br>

5. [Deploy the HPO task](#deploy)<br>

6. [Find best job results](#best)<br>

<a id = "setup"></a>
## Step 1: Setup


First, we must import the required modules. 

To use the WMLA API, we will be using the Python requests library.

In [2]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import json
import time
import urllib
import pandas as pd
import os,sys
import tarfile
import tempfile
from IPython.display import clear_output
import time
import pprint
import base64

# utility print function
def nprint(mystring) :
    print("**{}** : {}".format(sys._getframe(1).f_code.co_name,mystring))

# utility makedir
def makeDirIfNotExist(directory) :
    if not os.path.exists(directory):  
        nprint("Making directory {}".format(directory))
        os.makedirs(directory) 
    else :
        nprint("Directory {} already exists .. ".format(directory))


<a id = "configure"></a>
## Step 2: Configuring environment and project details

Provide your credentials in this cell, including your cluster url, username and password, and instance group.

In [3]:
def getconfig(cfg_in={}):
    cfg = {}
    #cfg["master_host"] = 'wmla-console-twmla.apps.wml1x180.ma.platformlab.ibm.com' # <=enter your host url here
    cfg["master_host"] = 'wmla-console-liqbj.apps.wml1x210.ma.platformlab.ibm.com'
    # ==== CLASS ENTER User login details below =====
    cfg["wmla_user"] = 'admin'  # <=enter your id here
    cfg["wmla_pwd"] = 'password'  # <=enter your pwd here
    #cfg["code_dir"] = "/home/wsuser/works/pytorch_hpo"
    cfg["data_path"] = 'pytorch_mnist' # <= enter the path of dataset

    # overwrite configs if passed
    for (k,v) in cfg_in.items() :
        nprint("Overriding Config {}:{} with {}".format(k,cfg[k],v))
        cfg[k] = v
    return cfg

# cfg is used as a global variable throughout this notebook
cfg=getconfig()

Here we will get and print out the API endpoints and setup requests session.   The following sections use the Watson ML Accelerator API to complete the various tasks required. We've given examples of a number of tasks but you should refer to the documentation at to see more details of what is possible and sample output you might expect.

    - https://www.ibm.com/support/knowledgecenter/SSFHA8_2.2.0/cm/deeplearning.html
   



In [4]:
# REST call variables


#commonHeaders={'accept': 'application/json'}
s=cfg["wmla_user"] + ":" + cfg["wmla_pwd"]
es = base64.b64encode(s.encode('utf-8')).decode("utf-8")
print(es)
commonHeaders={'Authorization': 'Basic '+es}


auth_url = 'https://{}/auth/v1/logon'.format(cfg["master_host"])
#print(auth_url)
auth_body = {'username': cfg["wmla_user"], 'password': cfg["wmla_pwd"]}
                            
a=requests.get(auth_url,headers=commonHeaders, verify=False, json=auth_body)
access_token=a.json()['accessToken']

#get api endpoint
def get_ep(mode="dl") :
    if mode=="dl" :
        dl_rest_url =  'https://' + cfg["master_host"] +'/platform/rest/deeplearning/v1'
        return dl_rest_url
    else :
        nprint("Error mode : {} not supported".format(mode))


print ("DL API Endpoints : {}".format(get_ep("dl")))

# Setup Requests session
commonHeaders={'accept': 'application/json', 'X-Auth-Token': access_token}
req = requests.Session()

YWRtaW46cGFzc3dvcmQ=
DL API Endpoints : https://wmla-console-liqbj.apps.wml1x210.ma.platformlab.ibm.com/platform/rest/deeplearning/v1


<a id = "health"></a>
## Step 3: Health Check

In this step, we will check if there are any existing HPO tasks and also verify the platform health.

Rest API: `GET platform/rest/deeplearning/v1/hypersearch`
- `Description`: Get all the HPO tasks that the login user can access.
- `OUTPUT`: A list of HPO tasks and each one with the same format which can be found in the api doc.

In [5]:
def hpo_health_check():
    getTuneStatusUrl = get_ep("dl") + '/hypersearch'
    nprint ('getTuneStatusUrl: %s' %getTuneStatusUrl)
    r = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, json=auth_body)
    
    if not r.ok:
        nprint('check hpo task status failed: code=%s, %s'%(r.status_code, r.content))
    else:
        if len(r.json()) == 0:
            nprint('There is no hpo task been created')
        for item in r.json():
            nprint('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))
            #print('Best:%s'%json.dumps(item.get('best'), sort_keys=True, indent=4))

hpo_health_check()


**hpo_health_check** : getTuneStatusUrl: https://wmla-console-liqbj.apps.wml1x210.ma.platformlab.ibm.com/platform/rest/deeplearning/v1/hypersearch
**hpo_health_check** : Hpo task: admin-hpo-5835408259613542, State: FINISHED
**hpo_health_check** : Hpo task: admin-hpo-5843973081207573, State: FINISHED
**hpo_health_check** : Hpo task: admin-hpo-5844576052554724, State: FINISHED
**hpo_health_check** : Hpo task: admin-hpo-6397378866788100, State: FINISHED
**hpo_health_check** : Hpo task: admin-hpo-6421166456056555, State: FINISHED
**hpo_health_check** : Hpo task: admin-hpo-6462458951734459, State: FINISHED
**hpo_health_check** : Hpo task: admin-hpo-6466373825972604, State: FINISHED
**hpo_health_check** : Hpo task: admin-hpo-6466845542476112, State: FINISHED


<a id = "train"></a>
## Step 4: Training with the HPO API


The WMLA framework requires 2 changes to your code to support the HPO API, and these are:

* Inject hyperparameters for the sub-training during search
* Retrieve sub-training result metric

Note that the code sections below show a comparison between the "before" and "HPO enabled" versions of the code by using `diff`.


1. Import the dependent libararies:

&nbsp;
&nbsp;
![image1](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_0.png)
&nbsp;
&nbsp;

2. Get the WMLA cluster `DLI_DATA_FS`, `RESULT_DIR` and `LOG_DIR` for the HPO training job. The `DLI_DATA_FS` can be used for shared data placement, the `RESULT_DIR` can be used for final model saving, and the `LOG_DIR` can be used for user logs and monitoring.

&nbsp;
**Note**: `DLI_DATA_FS` is set when installing the DLI cluster; `RESULT_DIR` and `LOG_DIR` is generated by WMLA for each HPO experiment.

&nbsp;
&nbsp;
![image1](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_1.png)
&nbsp;
&nbsp;

3. Replace the hyperparameter definition code by reading hyperparameters from the `config.json` file. the `config.json` is generated by WMLA HPO, which contains a set of hyperparameter candidates for each tuning jobs. The hyperparameters and the search space is defined when submitting the HPO task. For example, here the hyperparameter `learning_rate` is set to tune:

&nbsp;
&nbsp;
![image2](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_2.png)

&nbsp;
Then you could use the hyperparameter you get from `config.json` where you want:
&nbsp;
![image2](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_2_2.png)
&nbsp;
&nbsp;

4.  Write the tuning result into `val_dict_list.json` under `RESULT_DIR`. WMLA HPO will read this file for each tuning job to get the metric values. Define a `test_metrics` list to store all metric values and pass the epoch parameter to the test function. Then you can add the metric values to the `test_metrics` list during the training test process. Please note that the metric names should be specified when submitting the HPO task, and be consistent with the code here.
&nbsp;
For example, at the HPO task submit request, `loss` will be used as the objective metric the tuning will try to minimize the `loss`:

```
'algoDef': # Define the parameters for search algorithms  
{
    # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband  
    'algorithm': 'Random',   
    # Name of the target metric that we are trying to optimize when searching hyper-parameters.
    # It is the same metric name that the model update part 2 trying to dump.
    'objectiveMetric' : 'loss',
    # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
    # make the above objectiveMetric as small as possible, maximize means the opposite.
    'objective' : 'minimize',
    ...
}
```
&nbsp;
The code change:

&nbsp;
&nbsp;
![image2](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_3.png)
&nbsp;
&nbsp;

5. After the training completes, write the metric list into the `val_dict_list.json` file. 
&nbsp;
&nbsp;
![image2](https://github.com/IBM/wmla-learning-path/raw/dev/shared-images/hpo_update_model_5.png)
&nbsp;
&nbsp;




## Create script to pass to WMLA via API

This section creates a script external to this notebook which runs the batch jobs on the WMLA servers.

In [6]:
# Create a working directory to save our script in if it doesn't exist already

from pathlib import Path

working_directory = f'script_for_upload' 
model_main = f'WMLA_HPO_pytorch.py'

Path(working_directory).mkdir(exist_ok=True)

The next cell uses the writefile magic to create a script externally to this notebook in the location defined in working_directory above.

In [7]:
%%writefile {working_directory}/{model_main}

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# HPO - import dependent lib
import json
import os

# get dataset from DLI_DATA_FS
dataDir = os.environ["DLI_DATA_FS"]


if dataDir is not None:
    print("dataDir is: %s"%dataDir)
else:
    print("Warning: not found DATA_DIR from os env!")

model_path = os.environ["RESULT_DIR"]+"/model/saved_model"
print ("model_path: %s" %model_path)

# HPO - get hpo experiment hyper-parameter values from config.json
# The hyperparameters and the search space is defined when submitting the HPO task
# WMLA HPO will generate hpo experiment candidates and writes to config.json
try:
    hyper_params = json.loads(open("config.json").read())
    print('hyper_params: ', hyper_params)
    learning_rate = float(hyper_params.get("learning_rate", "0.01"))
except:
    print('failed to get hyper-parameters from config.json')
    learning_rate = 0.001
    pass


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


test_metrics = []
def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_metrics.append((epoch, {"loss": float(test_loss)}))


def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')

    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(dataDir, train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(dataDir, train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader, epoch)
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


    # HPO - dump metric values to val_dict_list.json start
    training_out =[]
    for test_metric in test_metrics:
        out = {'steps':test_metric[0]}
        for (metric,value) in test_metric[1].items():
            out[metric] = value
        training_out.append(out)
    with open('{}/val_dict_list.json'.format(os.environ['RESULT_DIR']), 'w') as f:
        json.dump(training_out, f)
# HPO - dump metric values to val_dict_list.json end
if __name__ == '__main__':
    main()



Overwriting script_for_upload/WMLA_HPO_pytorch.py


<a id = "deploy"></a>
## Step 5: Deploy the HPO task

Here we package up our model to send to the API for HPO.  



REST API: `POST /platform/rest/deeplearning/v1/hypersearch`

- Description: Start a new HPO task
- Content-type: Multi-Form
- Multi-Form Data:
  - files: Model file
  - form-filed: {‘data’: ‘String format of input parameters to start hpo task, let’s call it as **hpo_input** and show its specification later’}


#### Package model files for training

Package the updated model files into a tar file ending with `.modelDir.tar`

REST API expects a `modelDir.tar` with the model code inside ..


In [8]:
f = open(working_directory + "/" +model_main, 'rb')
files = {model_main : f}

#### Construct POST request data

**hpo_input** will be in Python `dict` or `json` format as shown below, and will convert to string when calling REST.

In [9]:
data =  {
        'modelSpec': # Define the model training related parameters
        {
         
            # These are the arguments we'll pass to the execution engine; they follow the same conventions
            # of the dlicmd.py command line launcher
            #
            # See:
            #   https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html
            # In this example, args after --model-dir are all the required parameter for the original model itself.
            #

               'args': '--exec-start PyTorch  --workerDeviceNum 1 --cs-datastore-meta type=fs,data_path={} \
                     --appName hpo-pytorch-mnist-gpu  --model-main {} --batch-size 64 --epochs 2 '.format(cfg["data_path"],model_main)  
        },
    
        'algoDef': # Define the parameters for search algorithms
        {
            # Name of the search algorithm, one of Random, Bayesian, Tpe, Hyperband, ExperimentGridSearch
            'algorithm': 'Random', 
            # Max running time of the hpo task in minutes, -1 means unlimited
            'maxRunTime': 60,  
            # Max number of training job to submitted for hpo task, -1 means unlimited’,
            'maxJobNum': 2,            
            # Max number of training job to run in parallel, default 1. It depends on both the
            # avaiable resource and if the search algorithm support to run in parallel, current only Random
            # fully supports to run in parallel, Hyperband and Tpe supports to to in parellel in some phase,
            # Bayesian runs in sequence now.
            'maxParalleJobNum': 2, 
            # Name of the target metric that we are trying to optimize when searching hyper-parameters.
            # It is the same metric name that the model update part 2 trying to dump.
            'objectiveMetric' : 'loss',
            # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
            # make the above objectiveMetric as small as possible, maximize means the opposite.
            'objective' : 'minimize',
        },
    
        # Define the hyper-paremeters to search and the corresponding search space.
        'hyperParams':
        [
             {
                 # Hyperparameter name, which will be the hyper-parameter key in config.json
                 'name': 'learning_rate',
                 # One of Range, Discrete
                 'type': 'Range',
                 # one of int, double, str
                 'dataType': 'DOUBLE',
                 # lower bound and upper bound when type=range and dataType=double
                 'minDbVal': 0.001,
                 'maxDbVal': 0.1,
                 # lower bound and upper bound when type=range and dataType=int
                 'minIntVal': 0,
                 'maxIntVal': 0,
                 # Discrete value list when type=discrete
                 'discreteDbVal': [],
                 'discreteIntVal': [],
                 'discreateStrVal': []
                 #step size to split the Range space. ONLY valid when type is Range
                 #'step': '0.002',
             }
         ]
    }
mydata={'data':json.dumps(data)}

#### Submit the Post request

Submit the HPO task through the Post call and an HPO name/id in string format will be returned.

In [10]:
def submit_job():
    startTuneUrl=get_ep('dl') + '/hypersearch'
    nprint("startTuneUrl : {}".format(startTuneUrl))
    nprint("files : {}".format(files))
    #nprint("myauth() : {}".format(auth_body))
    #print("hpo_job_id : {}".format(hpo_job_id))
    r = req.post(startTuneUrl, headers=commonHeaders, data=mydata, files=files, verify=False, json=auth_body)
    hpo_name=None
    if r.ok:
        hpo_name = r.json()
        print ('\nModel submitted successfully: {}'.format(hpo_name))
        
    else:
        print('\nModel submission failed with code={}, {}'. format(r.status_code, r.content))
    return hpo_name

hpo_job_id = submit_job()
print("hpo_job_id : {}".format(hpo_job_id))

**submit_job** : startTuneUrl : https://wmla-console-liqbj.apps.wml1x210.ma.platformlab.ibm.com/platform/rest/deeplearning/v1/hypersearch
**submit_job** : files : {'WMLA_HPO_pytorch.py': <_io.BufferedReader name='script_for_upload/WMLA_HPO_pytorch.py'>}

Model submitted successfully: {'uid': 'admin-hpo-6487177646872135', 'href': '/platform/rest/deeplearning/v1/hypersearch/admin-hpo-6487177646872135'}
hpo_job_id : {'uid': 'admin-hpo-6487177646872135', 'href': '/platform/rest/deeplearning/v1/hypersearch/admin-hpo-6487177646872135'}


Print out task details here.

In [11]:
def query_job_status(job_id,refresh_rate=3) :

    getHpoUrl = get_ep('dl')  +'/hypersearch/'+ hpo_job_id['uid']
    pp = pprint.PrettyPrinter(indent=2)

    keep_running=True
    res=None
    while(keep_running):
        res = req.get(getHpoUrl, headers=commonHeaders, verify=False, json=auth_body)
        experiments=res.json()['experiments']
        experiments = pd.DataFrame.from_dict(experiments)
        pd.set_option('max_colwidth', 120)
        clear_output()
        print("Refreshing every {} seconds".format(refresh_rate))
        display(experiments)
        pp.pprint(res.json())
        if(res.json()['state'] not in ['SUBMITTED','RUNNING']) :
            keep_running=False
        time.sleep(refresh_rate)
    return res
job_status = query_job_status(hpo_job_id,refresh_rate=10)

Refreshing every 10 seconds


,id,hyperParams,state,metricVal,appId,startTime,endTime
0,0,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.07682903454845375'}]",FINISHED,0.065198,liqbj-605,2020-11-17 19:59:48,2020-11-17 20:00:58
1,1,"[{'name': 'learning_rate', 'dataType': 'double', 'userDefined': False, 'fixedVal': '0.04433426721204368'}]",FINISHED,0.089596,liqbj-606,2020-11-17 19:59:53,2020-11-17 20:00:58


{ 'algoDef': { 'algorithm': 'Random',
               'maxJobNum': 2,
               'maxParalleJobNum': 2,
               'maxRunTime': 60,
               'objective': 'minimize',
               'objectiveMetric': 'loss'},
  'best': { 'appId': 'liqbj-605',
            'endTime': '2020-11-17 20:00:58',
            'hyperParams': [ { 'dataType': 'double',
                               'fixedVal': '0.07682903454845375',
                               'name': 'learning_rate',
                               'userDefined': False}],
            'id': 0,
            'metricVal': 0.06519806289672851,
            'startTime': '2020-11-17 19:59:48',
            'state': 'FINISHED'},
  'complete': 2,
  'createtime': '2020-11-17 19:59:46',
  'creator': 'admin',
  'duration': '00:01:00',
  'experiments': [ { 'appId': 'liqbj-605',
                     'endTime': '2020-11-17 20:00:58',
                     'hyperParams': [ { 'dataType': 'double',
                                        'fixedVal': '0

#### Notebook Complete 
Congratulations, you have completed our demonstration of using WMLA for distributed hyperparameter optimization search

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>